In [1]:
data_1 = spark.read.csv('s3://678-project/full_train.csv')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1588564790501_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
## rename columns
from functools import reduce
oldColumns = data_1.schema.names
newColumns = ['time_x',
 'date',
 'assetCode',
 'volume',
 'close',
 'open',
 'returnsClosePrevRaw1',
 'returnsOpenPrevRaw1',
 'returnsClosePrevMktres1',
 'returnsOpenPrevMktres1',
 'returnsClosePrevRaw10',
 'returnsOpenPrevRaw10',
 'returnsClosePrevMktres10',
 'returnsOpenPrevMktres10',
 'label',
 'time_y',
 'assetCodes',
 'urgency',
 'takeSequence',
 'marketCommentary',
 'companyCount',
 'relevance',
 'bodySize',
 'sentenceCount',
 'wordCount',
 'firstMentionSentence',
 'sentimentClass',
 'sentimentNegative',
 'sentimentNeutral',
 'sentimentPositive',
 'sentimentWordCount',
 'noveltyCount12H',
 'noveltyCount24H',
 'noveltyCount3D',
 'noveltyCount5D',
 'noveltyCount7D',
 'volumeCounts12H',
 'volumeCounts24H',
 'volumeCounts3D',
 'volumeCounts5D',
 'volumeCounts7D',
 'clean_headline']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df = reduce(lambda data_1, idx: data_1.withColumnRenamed(oldColumns[idx], newColumns[idx]), list(range(len(oldColumns))), data_1)
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- time_x: string (nullable = true)
 |-- date: string (nullable = true)
 |-- assetCode: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- close: string (nullable = true)
 |-- open: string (nullable = true)
 |-- returnsClosePrevRaw1: string (nullable = true)
 |-- returnsOpenPrevRaw1: string (nullable = true)
 |-- returnsClosePrevMktres1: string (nullable = true)
 |-- returnsOpenPrevMktres1: string (nullable = true)
 |-- returnsClosePrevRaw10: string (nullable = true)
 |-- returnsOpenPrevRaw10: string (nullable = true)
 |-- returnsClosePrevMktres10: string (nullable = true)
 |-- returnsOpenPrevMktres10: string (nullable = true)
 |-- label: string (nullable = true)
 |-- time_y: string (nullable = true)
 |-- assetCodes: string (nullable = true)
 |-- urgency: string (nullable = true)
 |-- takeSequence: string (nullable = true)
 |-- marketCommentary: string (nullable = true)
 |-- companyCount: string (nullable = true)
 |-- relevance: string (nullable = true)
 |-- bod

In [4]:
## remove first row
df = df.filter(df.time_x != 'time_x')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
stringIndexer = StringIndexer(inputCol="assetCode", outputCol="Index")
model = stringIndexer.fit(df)
indexed = model.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
encoder = OneHotEncoder(inputCol="Index", outputCol="categoryVec")
encoded = encoder.transform(indexed)
encoded.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- time_x: string (nullable = true)
 |-- date: string (nullable = true)
 |-- assetCode: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- close: string (nullable = true)
 |-- open: string (nullable = true)
 |-- returnsClosePrevRaw1: string (nullable = true)
 |-- returnsOpenPrevRaw1: string (nullable = true)
 |-- returnsClosePrevMktres1: string (nullable = true)
 |-- returnsOpenPrevMktres1: string (nullable = true)
 |-- returnsClosePrevRaw10: string (nullable = true)
 |-- returnsOpenPrevRaw10: string (nullable = true)
 |-- returnsClosePrevMktres10: string (nullable = true)
 |-- returnsOpenPrevMktres10: string (nullable = true)
 |-- label: string (nullable = true)
 |-- time_y: string (nullable = true)
 |-- assetCodes: string (nullable = true)
 |-- urgency: string (nullable = true)
 |-- takeSequence: string (nullable = true)
 |-- marketCommentary: string (nullable = true)
 |-- companyCount: string (nullable = true)
 |-- relevance: string (nullable = true)
 |-- bod

In [102]:
df = encoded.sample(False,1.0,0)
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5767260

In [103]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
market_numeric_cols = ['volume', 'close', 'open', 'returnsClosePrevRaw1', 'returnsOpenPrevRaw1',
                    'returnsClosePrevMktres1', 'returnsOpenPrevMktres1', 'returnsClosePrevRaw10',
                    'returnsOpenPrevRaw10', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10']
news_cols_numeric = ['urgency', 'takeSequence', 'wordCount', 'sentenceCount', 'companyCount',
                         'marketCommentary', 'relevance', 'sentimentNegative', 'sentimentNeutral',
                         'sentimentPositive', 'sentimentWordCount', 'noveltyCount12H', 'noveltyCount24H',
                         'noveltyCount3D', 'noveltyCount5D', 'noveltyCount7D', 'volumeCounts12H',
                         'volumeCounts24H', 'volumeCounts3D', 'volumeCounts5D', 'volumeCounts7D']
asset_code = ['categoryVec']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
label = ['label']
features = market_numeric_cols+news_cols_numeric
all = features+label+asset_code


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [106]:
# Write a custom function to convert the data type of DataFrame columns
def convertColumn(df, names, newType):
    for name in names:
        df = df.withColumn(name, df[name].cast(newType))
    return df

df = convertColumn(df,features+label,FloatType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [107]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
assembler = VectorAssembler(inputCols=features+asset_code, outputCol="features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
stages = []
stages += [assembler]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [110]:
new_df = df.select([column for column in all ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [111]:
cols = new_df.columns
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(new_df)

new_df = pipelineModel.transform(new_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
selectedCols = ['features','label']
new_df = new_df.select(selectedCols)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")
scaler = standardScaler.fit(new_df)
scaled_df = scaler.transform(new_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [114]:
train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [115]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol="label", maxIter=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [116]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [118]:
tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(),
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [119]:
import time

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [120]:
start = time.time()
model = tvs.fit(train_data)
end = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [121]:
print(end-start)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

316.9725344181061

In [122]:
predictions = model.transform(test_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
result = predictions\
    .select("label", "prediction")\
    .show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+
|       label|          prediction|
+------------+--------------------+
|-0.030424954| 0.07874768100064482|
|-0.044147052| 0.07832085022312935|
|  0.01645247| 0.09514988788393833|
|-0.079059266|  0.0754015513681755|
|-0.005511788| 0.07667727105087932|
|0.0010866148| 0.07800263296054188|
| -0.07961615| 0.09884413272473117|
|-0.014845077| 0.05085372398414559|
|-0.014845077|  0.0556733614125103|
|-0.014845077| 0.04223032487045959|
|-0.014845077| 0.04153866010687375|
|  0.02009334| 0.09075273255499727|
|  0.05652704| 0.06489157684276023|
|  0.05652704| 0.05240664448199269|
|  0.08049712|0.037560155731779316|
|  0.08049712| 0.03729936710075518|
|  0.11951588|0.033511518020113984|
| 0.038116235| 0.03864281555849686|
|  0.03255561|-0.01187779509177...|
| 0.066949144|-0.04610875822583...|
+------------+--------------------+
only showing top 20 rows

In [124]:
from pyspark.ml.evaluation import RegressionEvaluator
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="label",
                                predictionCol="prediction",
                                metricName="rmse")

rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 9.51055